# DETECCIÓN DE CÁNCER DE CUELLO UTERINO 

### Preprocesamiento de los registros clínicos

In [1]:
import pandas as pd
from pandas import DataFrame
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import KNNImputer

In [2]:
df_C = pd.read_csv('clinical.tsv', delimiter='\t')
df_C

,case_id,case_submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,cause_of_death_source,country_of_birth,country_of_residence_at_enrollment,days_to_birth,...,treatment_dose_units,treatment_duration,treatment_effect,treatment_effect_indicator,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_outcome_duration,treatment_type
0,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,'--,'--,'--,-23231,...,'--,'--,'--,'--,'--,Neoadjuvant,no,'--,'--,'--
1,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,'--,'--,'--,-23231,...,'--,'--,'--,'--,'--,Adjuvant,yes,Complete Response,'--,"Radiation Therapy, NOS"
2,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,'--,'--,'--,-23231,...,'--,'--,'--,'--,'--,'--,yes,Complete Response,'--,"Radiation Therapy, NOS"
3,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,'--,'--,'--,-23231,...,'--,'--,'--,'--,'--,Adjuvant,yes,Complete Response,'--,"Radiation, External Beam"
4,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,'--,'--,'--,-23231,...,'--,'--,'--,'--,'--,Adjuvant,yes,Complete Response,'--,"Brachytherapy, Low Dose"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932,ff95a7b3-ab96-4df1-95a5-6e50c3ff5e5f,HTMCP-03-06-02369,CGCI-HTMCP-CC,60,false,'--,'--,'--,'--,'--,...,'--,'--,'--,'--,'--,Adjuvant,unknown,'--,'--,"Radiation Therapy, NOS"
1933,ff95a7b3-ab96-4df1-95a5-6e50c3ff5e5f,HTMCP-03-06-02369,CGCI-HTMCP-CC,60,false,'--,'--,'--,'--,'--,...,'--,'--,'--,'--,'--,Neoadjuvant,no,'--,'--,'--
1934,ff95a7b3-ab96-4df1-95a5-6e50c3ff5e5f,HTMCP-03-06-02369,CGCI-HTMCP-CC,60,false,'--,'--,'--,'--,'--,...,'--,'--,'--,'--,'--,Adjuvant,unknown,'--,'--,"Pharmaceutical Therapy, NOS"
1935,ffd8d31f-bc4b-4e19-bbaf-0e26e9f3a107,TCGA-XS-A8TJ,TCGA-CESC,41,'--,'--,'--,'--,'--,-15298,...,'--,'--,'--,'--,'--,'--,yes,'--,'--,"Pharmaceutical Therapy, NOS"


#### Registros clínicos sin columnas vacías

In [3]:
df_C = df_C.loc[:, (df_C != "'--").any(axis=0)]
df_C

,case_id,case_submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,days_to_birth,days_to_death,ethnicity,gender,...,initial_disease_status,number_of_cycles,therapeutic_agents,treatment_anatomic_site,treatment_dose,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,-23231,'--,Unknown,female,...,Initial Diagnosis,'--,'--,'--,'--,'--,Neoadjuvant,no,'--,'--
1,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,-23231,'--,Unknown,female,...,'--,'--,'--,'--,'--,'--,Adjuvant,yes,Complete Response,"Radiation Therapy, NOS"
2,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,-23231,'--,Unknown,female,...,Initial Diagnosis,'--,'--,Unknown,45,'--,'--,yes,Complete Response,"Radiation Therapy, NOS"
3,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,-23231,'--,Unknown,female,...,'--,'--,'--,Pelvis,4500,'--,Adjuvant,yes,Complete Response,"Radiation, External Beam"
4,008d3744-e7f0-41a5-a419-702960cf1ccb,HTMCP-03-06-02025,CGCI-HTMCP-CC,63,false,'--,-23231,'--,Unknown,female,...,'--,'--,'--,Cervix,2500,'--,Adjuvant,yes,Complete Response,"Brachytherapy, Low Dose"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932,ff95a7b3-ab96-4df1-95a5-6e50c3ff5e5f,HTMCP-03-06-02369,CGCI-HTMCP-CC,60,false,'--,'--,'--,not hispanic or latino,female,...,'--,'--,'--,'--,'--,'--,Adjuvant,unknown,'--,"Radiation Therapy, NOS"
1933,ff95a7b3-ab96-4df1-95a5-6e50c3ff5e5f,HTMCP-03-06-02369,CGCI-HTMCP-CC,60,false,'--,'--,'--,not hispanic or latino,female,...,Initial Diagnosis,'--,'--,'--,'--,'--,Neoadjuvant,no,'--,'--
1934,ff95a7b3-ab96-4df1-95a5-6e50c3ff5e5f,HTMCP-03-06-02369,CGCI-HTMCP-CC,60,false,'--,'--,'--,not hispanic or latino,female,...,'--,'--,'--,'--,'--,'--,Adjuvant,unknown,'--,"Pharmaceutical Therapy, NOS"
1935,ffd8d31f-bc4b-4e19-bbaf-0e26e9f3a107,TCGA-XS-A8TJ,TCGA-CESC,41,'--,'--,-15298,'--,hispanic or latino,female,...,'--,'--,'--,'--,'--,'--,'--,yes,'--,"Pharmaceutical Therapy, NOS"


#### Información de las variables

In [4]:
df_C.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1937 entries, 0 to 1936
Data columns (total 53 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   case_id                        1937 non-null   object
 1   case_submitter_id              1937 non-null   object
 2   project_id                     1937 non-null   object
 3   age_at_index                   1937 non-null   object
 4   age_is_obfuscated              1937 non-null   object
 5   cause_of_death                 1937 non-null   object
 6   days_to_birth                  1937 non-null   object
 7   days_to_death                  1937 non-null   object
 8   ethnicity                      1937 non-null   object
 9   gender                         1937 non-null   object
 10  race                           1937 non-null   object
 11  vital_status                   1937 non-null   object
 12  year_of_birth                  1937 non-null   object
 13  yea

#### Eliminación de registros erróneos

In [5]:
df_C = df_C[df_C['tissue_or_organ_of_origin'] != 'Ovary']

In [6]:
df_C = df_C[df_C['icd_10_code'] != 'C56.9']

#### Conversión de la variable 'age_at_diagnosis' de días a años

In [7]:
df_C.loc[:,'age_at_diagnosis'] = pd.to_numeric(df_C['age_at_diagnosis'], errors='coerce')
df_C.loc[:,'age_at_diagnosis'] = df_C['age_at_diagnosis'] // 365.25

#### Eliminación de variables no relevantes

In [8]:
variables_no_relevantes = ['case_id', 'case_submitter_id', 'project_id', 'age_is_obfuscated', 'cause_of_death', 'days_to_birth', 
                           'days_to_death', 'gender', 'year_of_death', 'figo_staging_edition_year', 'last_known_disease_status', 
                           'method_of_diagnosis', 'prior_treatment', 'progression_or_recurrence', 'synchronous_malignancy', 
                           'tissue_or_organ_of_origin', 'year_of_diagnosis', 'chemo_concurrent_to_radiation']

df_C = df_C.drop(columns=variables_no_relevantes)
df_C

,age_at_index,ethnicity,race,vital_status,year_of_birth,age_at_diagnosis,ajcc_clinical_m,ajcc_clinical_n,ajcc_clinical_t,ajcc_pathologic_m,...,initial_disease_status,number_of_cycles,therapeutic_agents,treatment_anatomic_site,treatment_dose,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,63,Unknown,black or african american,Alive,'--,63.0,MX,NX,T2b,'--,...,Initial Diagnosis,'--,'--,'--,'--,'--,Neoadjuvant,no,'--,'--
1,63,Unknown,black or african american,Alive,'--,63.0,MX,NX,T2b,'--,...,'--,'--,'--,'--,'--,'--,Adjuvant,yes,Complete Response,"Radiation Therapy, NOS"
2,63,Unknown,black or african american,Alive,'--,63.0,MX,NX,T2b,'--,...,Initial Diagnosis,'--,'--,Unknown,45,'--,'--,yes,Complete Response,"Radiation Therapy, NOS"
3,63,Unknown,black or african american,Alive,'--,63.0,MX,NX,T2b,'--,...,'--,'--,'--,Pelvis,4500,'--,Adjuvant,yes,Complete Response,"Radiation, External Beam"
4,63,Unknown,black or african american,Alive,'--,63.0,MX,NX,T2b,'--,...,'--,'--,'--,Cervix,2500,'--,Adjuvant,yes,Complete Response,"Brachytherapy, Low Dose"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932,60,not hispanic or latino,black or african american,Alive,'--,NaN,'--,'--,T4,'--,...,'--,'--,'--,'--,'--,'--,Adjuvant,unknown,'--,"Radiation Therapy, NOS"
1933,60,not hispanic or latino,black or african american,Alive,'--,NaN,'--,'--,T4,'--,...,Initial Diagnosis,'--,'--,'--,'--,'--,Neoadjuvant,no,'--,'--
1934,60,not hispanic or latino,black or african american,Alive,'--,NaN,'--,'--,T4,'--,...,'--,'--,'--,'--,'--,'--,Adjuvant,unknown,'--,"Pharmaceutical Therapy, NOS"
1935,41,hispanic or latino,black or african american,Alive,1971,41.0,'--,'--,'--,M0,...,'--,'--,'--,'--,'--,'--,'--,yes,'--,"Pharmaceutical Therapy, NOS"


#### Conteo y sustitución de valores 'not reported',  'unknown' y '-- por 'NaN'

In [9]:
df_C = df_C.astype(str)
not_reported_counts = df_C.apply(lambda x: x.str.lower().eq('not reported').sum())
not_reported_counts

age_at_index                     0
ethnicity                      548
race                           212
vital_status                   134
year_of_birth                    0
age_at_diagnosis                 0
ajcc_clinical_m                  0
ajcc_clinical_n                  0
ajcc_clinical_t                  0
ajcc_pathologic_m                0
ajcc_pathologic_n                0
ajcc_pathologic_t                0
classification_of_tumor        612
days_to_diagnosis                0
days_to_last_follow_up           0
figo_stage                       0
icd_10_code                      0
morphology                       6
pregnant_at_diagnosis            0
primary_diagnosis                3
prior_malignancy                 0
site_of_resection_or_biopsy      9
tumor_grade                    746
days_to_treatment_end            0
days_to_treatment_start          0
initial_disease_status           0
number_of_cycles                 0
therapeutic_agents               0
treatment_anatomic_s

In [10]:
unknown_counts = df_C.apply(lambda x: x.str.lower().eq('unknown').sum())
unknown_counts

age_at_index                     0
ethnicity                      508
race                            24
vital_status                     0
year_of_birth                    0
age_at_diagnosis                 0
ajcc_clinical_m                  0
ajcc_clinical_n                  0
ajcc_clinical_t                  0
ajcc_pathologic_m                0
ajcc_pathologic_n                0
ajcc_pathologic_t                0
classification_of_tumor          6
days_to_diagnosis                0
days_to_last_follow_up           0
figo_stage                       0
icd_10_code                      0
morphology                       0
pregnant_at_diagnosis            0
primary_diagnosis                0
prior_malignancy                 0
site_of_resection_or_biopsy      0
tumor_grade                      0
days_to_treatment_end            0
days_to_treatment_start          0
initial_disease_status           0
number_of_cycles                 0
therapeutic_agents               0
treatment_anatomic_s

In [11]:
df_C.replace(['unknown', 'Unknown', 'Not Reported', 'not reported', "'--"], np.nan, inplace=True)
df_C

,age_at_index,ethnicity,race,vital_status,year_of_birth,age_at_diagnosis,ajcc_clinical_m,ajcc_clinical_n,ajcc_clinical_t,ajcc_pathologic_m,...,initial_disease_status,number_of_cycles,therapeutic_agents,treatment_anatomic_site,treatment_dose,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,63,NaN,black or african american,Alive,NaN,63.0,MX,NX,T2b,NaN,...,Initial Diagnosis,NaN,NaN,NaN,NaN,NaN,Neoadjuvant,no,NaN,NaN
1,63,NaN,black or african american,Alive,NaN,63.0,MX,NX,T2b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Adjuvant,yes,Complete Response,"Radiation Therapy, NOS"
2,63,NaN,black or african american,Alive,NaN,63.0,MX,NX,T2b,NaN,...,Initial Diagnosis,NaN,NaN,NaN,45,NaN,NaN,yes,Complete Response,"Radiation Therapy, NOS"
3,63,NaN,black or african american,Alive,NaN,63.0,MX,NX,T2b,NaN,...,NaN,NaN,NaN,Pelvis,4500,NaN,Adjuvant,yes,Complete Response,"Radiation, External Beam"
4,63,NaN,black or african american,Alive,NaN,63.0,MX,NX,T2b,NaN,...,NaN,NaN,NaN,Cervix,2500,NaN,Adjuvant,yes,Complete Response,"Brachytherapy, Low Dose"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932,60,not hispanic or latino,black or african american,Alive,NaN,nan,NaN,NaN,T4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Adjuvant,NaN,NaN,"Radiation Therapy, NOS"
1933,60,not hispanic or latino,black or african american,Alive,NaN,nan,NaN,NaN,T4,NaN,...,Initial Diagnosis,NaN,NaN,NaN,NaN,NaN,Neoadjuvant,no,NaN,NaN
1934,60,not hispanic or latino,black or african american,Alive,NaN,nan,NaN,NaN,T4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Adjuvant,NaN,NaN,"Pharmaceutical Therapy, NOS"
1935,41,hispanic or latino,black or african american,Alive,1971,41.0,NaN,NaN,NaN,M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,"Pharmaceutical Therapy, NOS"


#### Unificación de clases en 'primary_diagnosis'

In [12]:
df_C['primary_diagnosis'].value_counts()

primary_diagnosis
Squamous cell carcinoma, nonkeratinizing, NOS                734
Squamous cell carcinoma, NOS                                 414
Squamous cell carcinoma, keratinizing, NOS                   331
Adenocarcinoma, NOS                                          183
Squamous cell carcinoma, large cell, nonkeratinizing, NOS    102
Adenocarcinoma, endocervical type                             42
Mucinous adenocarcinoma, endocervical type                    34
Adenosquamous carcinoma                                       16
Neuroendocrine carcinoma, NOS                                 14
Basaloid squamous cell carcinoma                              11
Small cell carcinoma, NOS                                      9
Papillary squamous cell carcinoma                              9
Lymphoepithelial carcinoma                                     6
Endometrioid adenocarcinoma, NOS                               6
Glassy cell carcinoma                                          5
Squamot

In [13]:
df_C['primary_diagnosis'] = df_C['primary_diagnosis'].replace('Squamotransitional cell carcinoma', 'Squamous cell carcinoma, NOS')

In [14]:
df_C['primary_diagnosis'] = df_C['primary_diagnosis'].replace('Squamous cell carcinoma, large cell, nonkeratinizing, NOS', 'Squamous cell carcinoma, nonkeratinizing, NOS')

In [15]:
df_C['primary_diagnosis'] = df_C['primary_diagnosis'].replace('Tumor, NOS', np.nan)

In [16]:
df_C['primary_diagnosis'].value_counts()

primary_diagnosis
Squamous cell carcinoma, nonkeratinizing, NOS    836
Squamous cell carcinoma, NOS                     419
Squamous cell carcinoma, keratinizing, NOS       331
Adenocarcinoma, NOS                              183
Adenocarcinoma, endocervical type                 42
Mucinous adenocarcinoma, endocervical type        34
Adenosquamous carcinoma                           16
Neuroendocrine carcinoma, NOS                     14
Basaloid squamous cell carcinoma                  11
Small cell carcinoma, NOS                          9
Papillary squamous cell carcinoma                  9
Lymphoepithelial carcinoma                         6
Endometrioid adenocarcinoma, NOS                   6
Glassy cell carcinoma                              5
Carcinoma, NOS                                     4
Warty carcinoma                                    3
Name: count, dtype: int64

#### Unificación de clases en 'site_of_resection_or_biopsy'

In [17]:
df_C['site_of_resection_or_biopsy'].value_counts()

site_of_resection_or_biopsy
Cervix uteri                                            1825
Uterus, NOS                                               20
Lymph node, NOS                                           14
Lung, NOS                                                 12
Endocervix                                                 8
Liver                                                      7
Vagina, NOS                                                6
Connective, subcutaneous and other soft tissues, NOS       5
Abdomen, NOS                                               4
Ovary                                                      3
Peritoneum, NOS                                            3
Colon, NOS                                                 3
Specified parts of peritoneum                              2
Small intestine, NOS                                       2
Other ill-defined sites                                    2
Exocervix                                                

In [18]:
df_C['site_of_resection_or_biopsy'] = df_C['site_of_resection_or_biopsy'].replace('Specified parts of peritoneum', 'Peritoneum, NOS')

In [19]:
df_C['site_of_resection_or_biopsy'] = df_C['site_of_resection_or_biopsy'].replace('Other ill-defined sites', 'Other')

In [20]:
df_C['site_of_resection_or_biopsy'].value_counts()

site_of_resection_or_biopsy
Cervix uteri                                            1825
Uterus, NOS                                               20
Lymph node, NOS                                           14
Lung, NOS                                                 12
Endocervix                                                 8
Liver                                                      7
Vagina, NOS                                                6
Peritoneum, NOS                                            5
Connective, subcutaneous and other soft tissues, NOS       5
Abdomen, NOS                                               4
Colon, NOS                                                 3
Ovary                                                      3
Other                                                      2
Small intestine, NOS                                       2
Exocervix                                                  2
Pelvis, NOS                                              

#### Unificación de clases en 'therapeutic_agents'

In [21]:
df_C['therapeutic_agents'].value_counts()

therapeutic_agents
Cisplatin                  90
Paclitaxel                 36
Carboplatin                 3
Docetaxel                   3
Not Otherwise Specified     2
Name: count, dtype: int64

In [22]:
df_C['therapeutic_agents'] = df_C['therapeutic_agents'].replace('Not Otherwise Specified', np.nan)

In [23]:
df_C['therapeutic_agents'].value_counts()

therapeutic_agents
Cisplatin      90
Paclitaxel     36
Carboplatin     3
Docetaxel       3
Name: count, dtype: int64

#### Unificación de clases en 'ajcc_clinical_m' y 'ajcc_pathologic_m'

In [24]:
df_C['ajcc_clinical_m'].value_counts()

ajcc_clinical_m
MX    744
M0    308
M1     11
Name: count, dtype: int64

In [25]:
df_C['ajcc_pathologic_m'].value_counts()

ajcc_pathologic_m
MX    268
M0    232
M1     18
Name: count, dtype: int64

In [26]:
df_C['ajcc_clinical_m'] = df_C['ajcc_clinical_m'].replace(np.nan,'MX')

In [27]:
df_C['ajcc_pathologic_m'] = df_C['ajcc_pathologic_m'].replace(np.nan,'MX')

In [28]:
df_C['ajcc_clinical_m'].value_counts()

ajcc_clinical_m
MX    1615
M0     308
M1      11
Name: count, dtype: int64

In [29]:
df_C['ajcc_pathologic_m'].value_counts()

ajcc_pathologic_m
MX    1684
M0     232
M1      18
Name: count, dtype: int64

#### Unificación de clases en 'ajcc_clinical_n' y 'ajcc_pathologic_n'

In [30]:
df_C['ajcc_clinical_n'].value_counts()

ajcc_clinical_n
NX    756
N0    252
N1     80
Name: count, dtype: int64

In [31]:
df_C['ajcc_pathologic_n'].value_counts()

ajcc_pathologic_n
N0    268
NX    140
N1    120
Name: count, dtype: int64

In [32]:
df_C['ajcc_clinical_n'] = df_C['ajcc_clinical_n'].replace(np.nan,'NX')

In [33]:
df_C['ajcc_pathologic_n'] = df_C['ajcc_pathologic_n'].replace(np.nan,'NX')

In [34]:
df_C['ajcc_clinical_n'].value_counts()

ajcc_clinical_n
NX    1602
N0     252
N1      80
Name: count, dtype: int64

In [35]:
df_C['ajcc_pathologic_n'].value_counts()

ajcc_pathologic_n
NX    1546
N0     268
N1     120
Name: count, dtype: int64

#### Unificación de clases en 'ajcc_clinical_t' y 'ajcc_pathologic_t'

In [36]:
df_C['ajcc_clinical_t'].value_counts()

ajcc_clinical_t
T2b     479
T3b     342
T1b2    134
T2a2     57
T1b1     53
T4       48
T3a      34
T2a1     25
T2a       8
T1a2      5
Name: count, dtype: int64

In [37]:
df_C['ajcc_pathologic_t'].value_counts()

ajcc_pathologic_t
T1b1    146
T2b      82
T1b      72
T1b2     62
T3b      34
TX       34
T2a2     22
T4       20
T2a      18
T2a1     14
T2       12
T3        4
T3a       4
T1a1      2
Tis       2
Name: count, dtype: int64

In [38]:
df_C['ajcc_clinical_t'] = df_C['ajcc_clinical_t'].replace(np.nan,'TX')

In [39]:
df_C['ajcc_pathologic_t'] = df_C['ajcc_pathologic_t'].replace(np.nan,'TX')

In [40]:
df_C['ajcc_clinical_t'].value_counts()

ajcc_clinical_t
TX      749
T2b     479
T3b     342
T1b2    134
T2a2     57
T1b1     53
T4       48
T3a      34
T2a1     25
T2a       8
T1a2      5
Name: count, dtype: int64

In [41]:
df_C['ajcc_pathologic_t'].value_counts()

ajcc_pathologic_t
TX      1440
T1b1     146
T2b       82
T1b       72
T1b2      62
T3b       34
T2a2      22
T4        20
T2a       18
T2a1      14
T2        12
T3         4
T3a        4
T1a1       2
Tis        2
Name: count, dtype: int64

#### Comprobación de filas y columnas completamente vacías

In [42]:
print(f"Tamaño del dataset: {df_C.shape}")

filas_vacias = df_C[df_C.isna().all(axis=1)]
print(f"Número de filas vacías: {len(filas_vacias)}")

columnas_vacias = df_C.columns[df_C.isna().all(axis=0)]
print(f"Número de columnas vacías: {len(columnas_vacias)}")

Tamaño del dataset: (1934, 35)
Número de filas vacías: 0
Número de columnas vacías: 0


#### Conteo de celdas NaN para cada columna

In [43]:
df_C.isna().sum()

age_at_index                    138
ethnicity                      1056
race                            236
vital_status                    134
year_of_birth                  1326
age_at_diagnosis                  0
ajcc_clinical_m                   0
ajcc_clinical_n                   0
ajcc_clinical_t                   0
ajcc_pathologic_m                 0
ajcc_pathologic_n                 0
ajcc_pathologic_t                 0
classification_of_tumor         618
days_to_diagnosis              1319
days_to_last_follow_up         1438
figo_stage                      151
icd_10_code                    1322
morphology                        6
pregnant_at_diagnosis           761
primary_diagnosis                 6
prior_malignancy                137
site_of_resection_or_biopsy       9
tumor_grade                     774
days_to_treatment_end          1700
days_to_treatment_start        1691
initial_disease_status         1438
number_of_cycles               1830
therapeutic_agents          

#### Distinción de variables numéricas y categóricas

In [44]:
continuous_columns = ['age_at_index', 'year_of_birth', 'age_at_diagnosis', 'days_to_diagnosis', 'days_to_last_follow_up', 
                     'days_to_treatment_end', 'days_to_treatment_start', 'number_of_cycles', 'treatment_dose']

categorical_columns = ['ethnicity', 'race', 'vital_status','ajcc_clinical_m', 'ajcc_clinical_n', 
                       'ajcc_clinical_t', 'ajcc_pathologic_m', 'ajcc_pathologic_n', 'ajcc_pathologic_t', 
                       'classification_of_tumor', 'figo_stage', 'icd_10_code', 'morphology', 'pregnant_at_diagnosis',
                       'primary_diagnosis', 'prior_malignancy', 'site_of_resection_or_biopsy', 'tumor_grade', 
                       'initial_disease_status', 'therapeutic_agents', 'treatment_anatomic_site', 'treatment_frequency', 
                       'treatment_intent_type', 'treatment_or_therapy', 'treatment_outcome', 'treatment_type']

#### Codificación ordinal de columnas categóricas

In [45]:
encoder = OrdinalEncoder()

df_C_encoded = df_C.copy()
df_C_encoded[categorical_columns] = encoder.set_params(encoded_missing_value=-1).fit_transform(df_C[categorical_columns])

df_C_encoded = df_C_encoded.astype('float')
df_C_encoded = df_C_encoded.replace(-1, np.nan)
df_C_encoded

,age_at_index,ethnicity,race,vital_status,year_of_birth,age_at_diagnosis,ajcc_clinical_m,ajcc_clinical_n,ajcc_clinical_t,ajcc_pathologic_m,...,initial_disease_status,number_of_cycles,therapeutic_agents,treatment_anatomic_site,treatment_dose,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,63.0,NaN,2.0,0.0,NaN,63.0,2.0,2.0,6.0,2.0,...,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN
1,63.0,NaN,2.0,0.0,NaN,63.0,2.0,2.0,6.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,4.0
2,63.0,NaN,2.0,0.0,NaN,63.0,2.0,2.0,6.0,2.0,...,0.0,NaN,NaN,NaN,45.0,NaN,NaN,1.0,0.0,4.0
3,63.0,NaN,2.0,0.0,NaN,63.0,2.0,2.0,6.0,2.0,...,NaN,NaN,NaN,3.0,4500.0,NaN,0.0,1.0,0.0,5.0
4,63.0,NaN,2.0,0.0,NaN,63.0,2.0,2.0,6.0,2.0,...,NaN,NaN,NaN,0.0,2500.0,NaN,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932,60.0,1.0,2.0,0.0,NaN,NaN,2.0,2.0,9.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4.0
1933,60.0,1.0,2.0,0.0,NaN,NaN,2.0,2.0,9.0,2.0,...,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN
1934,60.0,1.0,2.0,0.0,NaN,NaN,2.0,2.0,9.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3.0
1935,41.0,0.0,2.0,0.0,1971.0,41.0,2.0,2.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.0


#### Mapeo de categorías a códigos numéricos

In [46]:
category_mapping = {}

for col, categories in zip(df_C_encoded[categorical_columns], encoder.categories_):
    
    valid_categories = [cat for cat in categories if pd.notna(cat)] # Exclusión de 'nan' como categoría válida
        
    mapping = dict(zip(range(len(valid_categories)), valid_categories))
    category_mapping[col] = mapping

for col, mapping in category_mapping.items():
    print(f"{col}: {mapping} \n")

ethnicity: {0: 'hispanic or latino', 1: 'not hispanic or latino'} 

race: {0: 'american indian or alaska native', 1: 'asian', 2: 'black or african american', 3: 'native hawaiian or other pacific islander', 4: 'white'} 

vital_status: {0: 'Alive', 1: 'Dead'} 

ajcc_clinical_m: {0: 'M0', 1: 'M1', 2: 'MX'} 

ajcc_clinical_n: {0: 'N0', 1: 'N1', 2: 'NX'} 

ajcc_clinical_t: {0: 'T1a2', 1: 'T1b1', 2: 'T1b2', 3: 'T2a', 4: 'T2a1', 5: 'T2a2', 6: 'T2b', 7: 'T3a', 8: 'T3b', 9: 'T4', 10: 'TX'} 

ajcc_pathologic_m: {0: 'M0', 1: 'M1', 2: 'MX'} 

ajcc_pathologic_n: {0: 'N0', 1: 'N1', 2: 'NX'} 

ajcc_pathologic_t: {0: 'T1a1', 1: 'T1b', 2: 'T1b1', 3: 'T1b2', 4: 'T2', 5: 'T2a', 6: 'T2a1', 7: 'T2a2', 8: 'T2b', 9: 'T3', 10: 'T3a', 11: 'T3b', 12: 'T4', 13: 'TX', 14: 'Tis'} 

classification_of_tumor: {0: 'Synchronous primary', 1: 'metastasis', 2: 'primary'} 

figo_stage: {0: 'Stage I', 1: 'Stage IA', 2: 'Stage IA1', 3: 'Stage IA2', 4: 'Stage IB', 5: 'Stage IB1', 6: 'Stage IB2', 7: 'Stage II', 8: 'Stage IIA',

#### Normalización de los registros

In [47]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_C_encoded)

df_C_scaled = pd.DataFrame(scaled_data, columns=df_C.columns)
df_C_scaled

,age_at_index,ethnicity,race,vital_status,year_of_birth,age_at_diagnosis,ajcc_clinical_m,ajcc_clinical_n,ajcc_clinical_t,ajcc_pathologic_m,...,initial_disease_status,number_of_cycles,therapeutic_agents,treatment_anatomic_site,treatment_dose,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,1.173213,NaN,-0.496028,-0.847088,NaN,1.269201,0.44216,0.439815,-0.569738,0.381384,...,-0.403607,NaN,NaN,NaN,NaN,NaN,1.777745,-1.544904,NaN,NaN
1,1.173213,NaN,-0.496028,-0.847088,NaN,1.269201,0.44216,0.439815,-0.569738,0.381384,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.562510,0.647290,-0.764546,0.549219
2,1.173213,NaN,-0.496028,-0.847088,NaN,1.269201,0.44216,0.439815,-0.569738,0.381384,...,-0.403607,NaN,NaN,NaN,-0.701523,NaN,NaN,0.647290,-0.764546,0.549219
3,1.173213,NaN,-0.496028,-0.847088,NaN,1.269201,0.44216,0.439815,-0.569738,0.381384,...,NaN,NaN,NaN,1.034427,0.872127,NaN,-0.562510,0.647290,-0.764546,1.512325
4,1.173213,NaN,-0.496028,-0.847088,NaN,1.269201,0.44216,0.439815,-0.569738,0.381384,...,NaN,NaN,NaN,-1.109658,0.165662,NaN,-0.562510,0.647290,-0.764546,-2.340099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,0.928355,0.248332,-0.496028,-0.847088,NaN,NaN,0.44216,0.439815,0.571706,0.381384,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.562510,NaN,NaN,0.549219
1930,0.928355,0.248332,-0.496028,-0.847088,NaN,NaN,0.44216,0.439815,0.571706,0.381384,...,-0.403607,NaN,NaN,NaN,NaN,NaN,1.777745,-1.544904,NaN,NaN
1931,0.928355,0.248332,-0.496028,-0.847088,NaN,NaN,0.44216,0.439815,0.571706,0.381384,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.562510,NaN,NaN,-0.413887
1932,-0.622418,-4.026871,-0.496028,-0.847088,0.763753,-0.561800,0.44216,0.439815,0.952187,-2.679180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.647290,NaN,-0.413887


#### Imputación de valores 'NaN' mediante KNN

In [48]:
imputer = KNNImputer(n_neighbors=10)
imputed_data = imputer.fit_transform(scaled_data)

#### Desnormalización de los registros

In [49]:
imputed_data = scaler.inverse_transform(imputed_data)

df_C_imputed = pd.DataFrame(imputed_data, columns=df_C.columns)
df_C_imputed

,age_at_index,ethnicity,race,vital_status,year_of_birth,age_at_diagnosis,ajcc_clinical_m,ajcc_clinical_n,ajcc_clinical_t,ajcc_pathologic_m,...,initial_disease_status,number_of_cycles,therapeutic_agents,treatment_anatomic_site,treatment_dose,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,63.0,1.000000e+00,2.0,0.0,1944.0,63.0,2.0,2.0,6.0,2.000000e+00,...,0.0,4.8,1.8,1.600000e+00,43.0,0.1,1.0,0.0,9.000000e-01,3.6
1,63.0,1.000000e+00,2.0,0.0,1947.8,63.0,2.0,2.0,6.0,2.000000e+00,...,0.0,3.9,1.2,2.600000e+00,1428.1,0.1,0.0,1.0,0.000000e+00,4.0
2,63.0,1.000000e+00,2.0,0.0,1947.8,63.0,2.0,2.0,6.0,2.000000e+00,...,0.0,4.2,1.0,2.200000e+00,45.0,0.1,0.0,1.0,0.000000e+00,4.0
3,63.0,1.000000e+00,2.0,0.0,1945.8,63.0,2.0,2.0,6.0,2.000000e+00,...,0.0,4.2,1.2,3.000000e+00,4500.0,0.1,0.0,1.0,0.000000e+00,5.0
4,63.0,1.000000e+00,2.0,0.0,1949.9,63.0,2.0,2.0,6.0,2.000000e+00,...,0.1,3.9,1.4,2.220446e-16,2500.0,0.1,0.0,1.0,0.000000e+00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,60.0,1.000000e+00,2.0,0.0,1953.0,49.3,2.0,2.0,9.0,2.000000e+00,...,0.6,3.8,1.2,2.400000e+00,538.9,0.1,0.0,1.0,1.600000e+00,4.0
1930,60.0,1.000000e+00,2.0,0.0,1949.7,43.2,2.0,2.0,9.0,2.000000e+00,...,0.0,4.3,1.6,1.700000e+00,42.3,0.1,1.0,0.0,7.000000e-01,3.4
1931,60.0,1.000000e+00,2.0,0.0,1953.0,43.2,2.0,2.0,9.0,2.000000e+00,...,0.7,3.8,1.2,1.700000e+00,39.2,0.1,0.0,0.9,1.800000e+00,3.0
1932,41.0,-1.110223e-16,2.0,0.0,1971.0,41.0,2.0,2.0,10.0,2.220446e-16,...,0.0,3.3,1.0,1.600000e+00,477.5,0.0,0.0,1.0,1.110223e-16,3.0


#### Conversión de variables en formato 'float' a 'int'

In [50]:
df_C_imputed = df_C_imputed.astype(int)
df_C_imputed

,age_at_index,ethnicity,race,vital_status,year_of_birth,age_at_diagnosis,ajcc_clinical_m,ajcc_clinical_n,ajcc_clinical_t,ajcc_pathologic_m,...,initial_disease_status,number_of_cycles,therapeutic_agents,treatment_anatomic_site,treatment_dose,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,63,1,2,0,1944,63,2,2,6,2,...,0,4,1,1,43,0,1,0,0,3
1,63,1,2,0,1947,63,2,2,6,2,...,0,3,1,2,1428,0,0,1,0,4
2,63,1,2,0,1947,63,2,2,6,2,...,0,4,1,2,45,0,0,1,0,4
3,63,1,2,0,1945,63,2,2,6,2,...,0,4,1,3,4500,0,0,1,0,5
4,63,1,2,0,1949,63,2,2,6,2,...,0,3,1,0,2500,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,60,1,2,0,1953,49,2,2,9,2,...,0,3,1,2,538,0,0,1,1,4
1930,60,1,2,0,1949,43,2,2,9,2,...,0,4,1,1,42,0,1,0,0,3
1931,60,1,2,0,1953,43,2,2,9,2,...,0,3,1,1,39,0,0,0,1,3
1932,41,0,2,0,1971,41,2,2,10,0,...,0,3,1,1,477,0,0,1,0,3


#### Comprobación de la correcta imputación

In [51]:
df_C_imputed.isna().sum()

age_at_index                   0
ethnicity                      0
race                           0
vital_status                   0
year_of_birth                  0
age_at_diagnosis               0
ajcc_clinical_m                0
ajcc_clinical_n                0
ajcc_clinical_t                0
ajcc_pathologic_m              0
ajcc_pathologic_n              0
ajcc_pathologic_t              0
classification_of_tumor        0
days_to_diagnosis              0
days_to_last_follow_up         0
figo_stage                     0
icd_10_code                    0
morphology                     0
pregnant_at_diagnosis          0
primary_diagnosis              0
prior_malignancy               0
site_of_resection_or_biopsy    0
tumor_grade                    0
days_to_treatment_end          0
days_to_treatment_start        0
initial_disease_status         0
number_of_cycles               0
therapeutic_agents             0
treatment_anatomic_site        0
treatment_dose                 0
treatment_

#### Creación de un nuevo csv con los registros imputados y codificados

In [53]:
df_C_imputed.to_csv('df_clinical.csv', sep=';')